In [1]:
import nltk

In [2]:
from nltk.corpus import twitter_samples

In [3]:
nltk.download("twitter_samples")

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [4]:
text = twitter_samples.strings()

In [5]:
len(text)

30000

In [6]:
text[0]

'hopeless for tmr :('

In [7]:
import re

In [8]:
import string

In [9]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords

In [11]:
stopword = stopwords.words("english")

In [12]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [14]:
!pip install emoji

In [15]:
import emoji

In [16]:
lines = []
for i in range(len(text)):
  if type(text[i]) == "str":
    a = text[i]
  elif type(text[i]) != "str":
    a = str(text[i])
  a = a.lower()
  a = re.sub("https?://\S+|www\.\S+", "", a)
  a = re.sub("#\w+", "", a)
  a = re.sub("@\w+", "", a)
  a = re.sub("\d+", "", a)
  a = re.sub(":\(|:\)|:\-\(|\(+|\)+", "", a)
  a = emoji.replace_emoji(a, "")
  word = word_tokenize(a)
  for i in word:
    if i in string.punctuation:
      a.replace(i, "")
  for b in sent_tokenize(a):
    lines.append(b.strip())
print(len(lines))

44754


In [ ]:
lines

In [18]:
def word_count(word):
  count= 0
  for i in lines:
    if word in i:
      count += 1
  return count

In [19]:
from collections import Counter

In [ ]:
c = []
for i in lines:
  a = word_tokenize(i)
  for j in a:
    if j in string.punctuation:
      a.remove(j)
  c.extend(a)
counter = Counter(c)
counter

In [21]:
len(counter)

19237

In [22]:
import pandas as pd

In [23]:
a = []
b = []
for i,j in zip(counter.keys(), counter.values()):
  a.append(i)
  b.append(j)
frame = {"words" : a, "count" : b}
frame = pd.DataFrame(frame)

In [24]:
N_class = 0
for i in frame["count"]:
  N_class += i
c = []
for i in range(len(frame["count"])):
  c.append(frame["count"][i]/N_class)
frame["probab"] = c
frame["count"] = frame["count"].astype("int16")
frame["probab"] = frame["probab"].astype("float16")

In [25]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   words   19237 non-null  object 
 1   count   19237 non-null  int16  
 2   probab  19237 non-null  float16
dtypes: float16(1), int16(1), object(1)
memory usage: 225.6+ KB


In [ ]:
frame["words"]

In [ ]:
frame.head(20)

In [28]:
line = lines[0]
line

'hopeless for tmr'

In [29]:
def delete(word):
  delete = []
  for i in range(len(word)):
    L = word[:i]
    R = word[i+1:]
    a = L+R
    delete.append(a)
  return delete

In [30]:
def insert(word):
  insert = []
  letters = "abcdefghijklmnopqrstuvwxyz"
  for i in range(len(word)+1):
    for j in range(len(letters)):
      L = word[:i]
      R = word[i:]
      a = L + letters[j] + R
      insert.append(a)
  return insert

In [31]:
def switch(word):
  def swap(word,a,b):
    word_list = list(word)
    word_list[a], word_list[b] = word_list[b], word_list[a]
    return "".join(word_list)
  switch = []
  for i in range(len(word)-1):
    a = swap(word, i, i+1)
    switch.append(a)
  return switch

In [32]:
def replace(word):
  replace = []
  letters = "abcdefghijklmnopqrstuvwxyz"
  for i in range(len(word)):
    for j in range(len(letters)):
      a = word[:i] + letters[j] + word[i+1:]
      replace.append(a)
  return replace

In [33]:
def one_edits(word):
  one_edit = set()
  one_edit.update(delete(word))
  one_edit.update(switch(word))
  one_edit.update(insert(word))
  one_edit.update(replace(word))
  return one_edit

In [34]:
def delete_two_edits(word):
  delete = []
  for i in range(len(word)-1):
    L = word[:i]
    R = word[i+2:]
    a = L+R
    delete.append(a)
  return delete

In [35]:
def insert_two_words(word):
  insert = []
  letters = "abcdefghijklmnopqrstuvwxyz"
  for i in range(len(word)+1):
      L = word[:i]
      R = word[i:]
      for j in letters:
        for k in letters:
          a = L + j + k+ R
          insert.append(a)
  return insert

In [36]:
def replace_two_words(word):
  replace = []
  letters = "abcdefghijklmnopqrstuvwxyz"
  for i in range(len(word)-1):
    for j in letters:
      for k in letters:
        a = word[:i] + j + k + word[i+2:]
        replace.append(a)
  return replace

In [37]:
def switch_two_words(word):
  switches = []
  for i in word:
    a = switch(i)
    for  j in a:
      switches.append(j)
  return switches

In [38]:
def edit_two_words(word):
  edit_two_word = set()
  edit_two_word.update(delete_two_edits(word))
  edit_two_word.update(insert_two_words(word))
  edit_two_word.update(replace_two_words(word))
  edit_two_word.update(switch_two_words(word))
  return edit_two_word

In [39]:
import numpy as np

In [40]:
def autocorrect(word):
  def minimum_edit_distance(source, target):
    def cost(source,target,row,column):
      if source[row] != target[column]:
        return 2
      elif source[row] == target[column]:
        return 0
    source  = " " + source
    target  = " " + target
    D = np.zeros((len(source), len(target)))
    for row in range(len(source)):
      D[row,0] = row
    for column in range(len(target)):
      D[0,column] = column
    for column in range(1,len(target)):
      for row in range(1,len(source)):
        D[row,column] = min((D[row-1,column]+1),(D[row,column-1]+1),(D[row-1,column-1]+cost(source,target,row,column)))
    return int(D[-1,-1])
  corrections = dict()
  if word in frame["words"]:
    return f"{word} : probablity = 100%"
  elif word not in frame["words"]:
    a = set(frame["words"])
    j = list(a.intersection(one_edits(word)))
    k = list(a.intersection(edit_two_words(word)))
    c = []
    for i in j:
      c.append(i)
    for i in k:
      c.append(i)
      corrections["expected_words"] = []
      corrections["probablity"] = []
      correction = pd.DataFrame(corrections)
      correction["expected_words"] = c
      d = list(frame["words"])
      e = []
      for i in c:
        index = d.index(i)
        e.append(frame["probab"][index])
      correction["probablity"] = e
      correction["probablity"] = correction["probablity"].astype("float32")
      words = set(sorted(c, reverse= True))
      f = dict()
      for i in c:
        a = minimum_edit_distance(word, i)
        f[i] = a
      e = sorted(list(f.values()))
      g = list(f.keys())
      h = []
      for i in e[:3]:
        index = list(f.values()).index(i)
        h.append(g[index])
      n = set(h)
      m = set.intersection(n,words)
      return m

In [54]:
autocorrect("stdent")

{'strident', 'student'}